In [6]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [7]:
# Load the image using Pillow
img = Image.open('Mycelium_model_real_image_1.png').convert('L')  # 'L' converts to grayscale

# Convert image to NumPy array and binarize (thresholding at 127)
img_array = np.array(img)
threshold = 127
binary_img = np.where(img_array > threshold, 1, 0)  # Binarize image

# Convert to PyTorch tensor
skeleton = torch.tensor(binary_img, dtype=torch.float32).to(device)

In [8]:
# Hyperparameters
gridSize = skeleton.shape[0]
totalIterations = 500
substrate_val = 0.3
depth = 2
alpha_mat = torch.zeros((gridSize, gridSize)).to(device)

# Reaction-Diffusion parameters
pa = 0.5; pb = 0.8; pc = 0.16; pe = 2.6; d = 30; dt = 1e-1
threshold = 0.5; pk = 0.05; gamma = 625; ph = 1
amax = 100; smax = 35

# Laplacian weights
lap_side = 0.35
lap_diag = 0.1
lap = 1 / 9

In [9]:
# Initial conditions for u, v, and c
u = torch.zeros((gridSize, gridSize)).to(device)
v = torch.zeros((gridSize, gridSize)).to(device)
c = torch.zeros((gridSize, gridSize)).to(device)
mid = gridSize // 2
c_new = torch.zeros_like(c)
u_new = torch.zeros_like(u)
v_new = torch.zeros_like(v)
f_uv = torch.zeros_like(u)
g_uv = torch.zeros_like(v)

# Creating a gradient for substrate matrix 'n'
n = torch.linspace(1, substrate_val, gridSize).repeat(gridSize, 1).to(device)
n[skeleton != 1] = -1  # Apply the skeleton constraints

In [10]:
# Initialize activator (u), inhibitor (v), and mycelium (c) in a small circular region in the center
for k in range(-5, 6):
    for j in range(mid - k, mid + k + 1):
        for l in range(mid - k, mid + k + 1):
            u[j, l] = 0.5 + (torch.rand(1).item() / (0.5 * n[j, l]))
            v[j, l] = 0.1 + (torch.rand(1).item() / (0.5 * n[j, l]))
    c[mid - k:mid + k + 1, mid - k + k + 1] = 1.0

ij_mat = torch.zeros((gridSize, gridSize)).to(device)

In [ ]:
# Main simulation loop
for steps in range(totalIterations):
    # Calculate reaction terms
    f_uv = (pa * u + (u ** 2) - pb * u * v) * n
    g_uv = pe * u ** 3 - v

    # Calculate Laplacian (using finite difference)
    for i in range(depth, gridSize - depth):
        for j in range(depth, gridSize - depth):
            if c[i, j] > 0:
                for dx in range(-depth, depth + 1):
                    for dy in range(-depth, depth + 1):
                        if dx == 0 and dy == 0:
                            ij_mat[i, j] = lap  # Center cell
                        elif abs(dx) == abs(dy):
                            ij_mat[i + dx, j + dy] = lap_diag  # Diagonal cells
                        else:
                            ij_mat[i + dx, j + dy] = lap_side  # Side cells

    # Update concentrations
    for i in range(1, gridSize - 1):
        for j in range(1, gridSize - 1):
            v_new[i, j] = v[i, j] + dt * ij_mat[i, j] * (d * (
                    lap_side * (v[i + 1, j] + v[i - 1, j] + v[i, j + 1] + v[i, j - 1])
                    + lap_diag * (v[i - 1, j - 1] + v[i + 1, j - 1] + v[i - 1, j + 1] + v[i + 1, j + 1])
                    - lap * v[i, j]) + gamma * g_uv[i, j])

            u_new[i, j] = u[i, j] + dt * ij_mat[i, j] * (
                    lap_side * (u[i + 1, j] + u[i - 1, j] + u[i, j + 1] + u[i, j - 1])
                    + lap_diag * (u[i - 1, j - 1] + u[i + 1, j - 1] + u[i - 1, j + 1] + u[i + 1, j + 1])
                    - lap * u[i, j] + gamma * f_uv[i, j])

            # Boundary conditions and constraints
            if n[i, j] == -1:
                u_new[i, j] = u[i, j] - torch.rand(1).item() / 10000

            if u[i, j] <= threshold:
                alpha = 0.49
            else:
                alpha = 0.49 - 2.5 * (u[i, j] - threshold)
                v_new[i, j] = 0

            c_new[i, j] = c[i, j] + dt * gamma * ph * c[i, j] * (alpha - c[i, j]) * (c[i, j] - 1)

            # Limiters for activator, inhibitor, and mycelium
            u_new[i, j] = torch.clamp(u_new[i, j], 0, amax)
            v_new[i, j] = torch.clamp(v_new[i, j], 0, smax)
            c_new[i, j] = torch.clamp(c_new[i, j], 0, 1)

    # Update matrices
    u = u_new.clone()
    v = v_new.clone()
    c = c_new.clone()

    # Visualization every 20 iterations
    if steps % 20 == 0:
        plt.subplot(2, 2, 1)
        plt.imshow(u.cpu().numpy(), cmap='jet', vmin=0, vmax=amax)
        plt.title(f'Activator u at iteration {steps}')
        plt.colorbar()

        plt.subplot(2, 2, 2)
        plt.imshow(v.cpu().numpy(), cmap='jet', vmin=0, vmax=smax)
        plt.title(f'Inhibitor v at iteration {steps}')
        plt.colorbar()

        plt.subplot(2, 2, 3)
        plt.imshow(c.cpu().numpy(), cmap='jet', vmin=0, vmax=1)
        plt.title(f'Mycelium at iteration {steps}')
        plt.colorbar()

        plt.subplot(2, 2, 4)
        plt.imshow(n.cpu().numpy(), cmap='jet', vmin=torch.min(n).item(), vmax=torch.max(n).item())
        plt.title('Substrate')
        plt.colorbar()

        plt.tight_layout()
        plt.show()